# My house modeling

["Planimetria di riferimento"](https://github.com/CostanzaBrachetti/ggpl/blob/master/2016-12-02/esempio.jpg)

Tramite inkscape genero 5 file, uno per le linee dei muri esterni, uno per le linee dei muri interni, uno per le porte interne, uno per quella esterna, ed infine uno per le finestre.
Il metodo creaPlanimetria si occupa di prendere il file della struttura esterna e di crearla, successivamente richiama tutti gli altri metodi come strutturaInterna() (che si occupa,dato il file delle linee della struttura interna, di costruirla e restituirla). Allo stesso modo operano i metodi porteInterne(), creaFinestra(), creaPortaEsterna(). 
Per quanto riguarda le porte e le finestre una volta create viene fatta la DIFFERENCE per ricavare lo spazio dove poterle inserire. 
per quanto riguarda il pavimento ho utilizzato le due liste di punti FV e V per poter richiamare il metodo EXPLODE per collegare tutti i vertici della base. Tuttavia ottenevo un difetto che ho dovuto rimuovere manualmente con l'ausilio di una struttura, chiamata appunto difetto, che mi è servita a fare la difference su un pezzo di pavimento di troppo. 

In [ ]:
from pyplasm import*
from larlib import*
import csv

In [79]:
def stringToFloat(str):
    l = str.split(',')
    newList=[]
    for c in l:
        elem = float(c)/20.
        newList.append(elem)
    return newList

In [80]:
def porteInterne(file_name):
    porte = STRUCT([QUOTE([0])])
    with open(file_name,'r') as csvFile:
        reader = csv.reader(csvFile, delimiter=' ', quotechar='|')
        for row in reader:
            l = stringToFloat(row[0])
            riga = MKPOL([[[l[0],l[1],0],[l[2],l[3],0]],[[1,2]],1])
            porte = STRUCT([porte,riga])
        porte = OFFSET([0.15,0.15,0])(porte)
        porte = OFFSET([-0.10,0,2])(porte)
    return porte

In [81]:
def strutturaInterna(file_name):
    interni = STRUCT([QUOTE([0])])
    with open(file_name,'r') as csvFile:
        reader = csv.reader(csvFile, delimiter=' ', quotechar='|')
        for row in reader:
            l = stringToFloat(row[0])
            riga = MKPOL([[[l[0],l[1],0],[l[2],l[3],0]],[[1,2]],1])
            interni = STRUCT([interni,riga])
        interni = OFFSET([0.1,0.1,2.5])(interni)
    return interni

In [83]:
def creaFinestre(file_name): 
    spazioFinestre = STRUCT([QUOTE([0])])
    with open(file_name,'r') as csvFile:
        reader = csv.reader(csvFile, delimiter=' ', quotechar='|')
        for row in reader:
            l = stringToFloat(row[0])
            riga = MKPOL([[[l[0],l[1],0],[l[2],l[3],0]],[[1,2]],1])
            spazioFinestre = STRUCT([spazioFinestre,riga])  
    spazioFinestre = T(3)(0.7)(spazioFinestre)
    spazioFinestre = OFFSET([-0.02,-0.01,0])(spazioFinestre)
    spazioFinestre = OFFSET([0.3,0.3,1.3])(spazioFinestre)
    return spazioFinestre

In [84]:
def creaPortaEsterna(file_name):
    porta = STRUCT([QUOTE([0])])
    with open(file_name,'r') as csvFile:
        reader = csv.reader(csvFile, delimiter=' ', quotechar='|')
        for row in reader:
            l = stringToFloat(row[0])
            riga = MKPOL([[[l[0],l[1],0],[l[2],l[3],0]],[[1,2]],1])
            porta = STRUCT([porta,riga]) 
    porta = OFFSET([0.32,0.1,2])(porta)
    pomello = SPHERE(0.03)([100,100])
    pomello = T([1,2,3])([l[0],l[1],1.4])(pomello)
    return porta

In [85]:
def lunghezzaCella(lista):
    newLista = []
    for i in range(len(lista)):
        newLista.append(i+1)
    return newLista    

In [86]:
def creaPavimento(file_name):
    pavimento = STRUCT([QUOTE([0])])
    with open(file_name,'r') as csvFile:
        reader = csv.reader(csvFile, delimiter=' ', quotechar='|')
        for row in reader:
            l = stringToFloat(row[0])
            riga = MKPOL([[[l[0],l[1],0],[l[2],l[3],0]],[[1,2]],1])
            pavimento = STRUCT([pavimento,riga])
    listaPunti = UKPOL(pavimento)
    V = listaPunti[0]
    EV = listaPunti[1]
    FV = []
    for i in range(1,len(V)):
        FV.append(i)
    FV = [FV]
    pavimento = EXPLODE(1,1,1)(MKPOLS((V,FV)))
    pavimento = OFFSET([0,0,0.1])(pavimento)
    V = [[0,0],[15,22],[0,22]]
    EV = [[0,2],[2,1],[1,0]]
    FV = [[0,1,2]]
    difetto = EXPLODE(1.2,1.2,1.2)(MKPOLS((V,FV)))
    difetto = T([1,2])([21,0.8])(difetto)
    difetto = OFFSET([0,0,0.1])(difetto)
    
    pavimento = DIFFERENCE([pavimento,difetto])
    return pavimento

In [87]:
def creaPlanimetria(file_name):
    planimetria = STRUCT([QUOTE([0])])
    with open(file_name,'r') as csvFile:
        reader = csv.reader(csvFile, delimiter=' ', quotechar='|')
        for row in reader:
            l = stringToFloat(row[0])
            riga = MKPOL([[[l[0],l[1],0],[l[2],l[3],0]],[[1,2]],1])
            planimetria= STRUCT([planimetria,riga])
    planimetria = OFFSET([0.3,0.3,2.5])(planimetria)
    interni = strutturaInterna("interni.lines")
    spazioFinestre = creaFinestre("finestre.lines")
    porte = porteInterne("porteInterne.lines")
    pavimento = creaPavimento("muriEsterni.lines")
    portaEsterna = creaPortaEsterna("portaEsterna.lines")
    interni = DIFFERENCE([interni,porte])
    planimetria = DIFFERENCE([planimetria,spazioFinestre])
    planimetria = DIFFERENCE([planimetria,portaEsterna])
    portaEsterna = COLOR(Color4f([128/255.,0,0,1]))(portaEsterna)
    spazioFinestre = COLOR(Color4f([0,1,1,1]))(spazioFinestre)
    interni = COLOR(Color4f([205/255.,127/255.,50/255.,1]))(interni)
    planimetria = COLOR(Color4f([205/255.,127/255.,50/255.,1]))(planimetria)
    pavimento = COLOR(Color4f([1,228/255.,196/255.,1]))(pavimento)
    planimetria = STRUCT([planimetria,interni,spazioFinestre,pavimento,portaEsterna])
    VIEW(planimetria)

In [88]:
creaPlanimetria("muriEsterni.lines")
#creaFinestre("finestre.lines")
#creaPavimento("muriEsterni.lines")

Child 3
